In [345]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import QED
from rdkit.Contrib.SA_Score import sascorer
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from rdkit.Chem import Descriptors
from rdkit.Chem.Lipinski import NumHDonors, NumHAcceptors
from rdkit.Chem.rdMolDescriptors import CalcNumRotatableBonds
import xgboost as xgb

In [347]:
df = pd.read_csv(r"C:\Users\kiril\Desktop\10.2 семестр\хакатон спб\HDAC6_smiles_10k.csv")

In [348]:
df

,SMILES
0,O=C(CNCCNc1ccc2c(c1)/C(=C1\Nc3ccccc31)C(=O)N2)...
1,C=c1[nH]c(=O)/c(=C(C)/N=C(\S)Nc2ccc(CCNC3c4cc(...
2,CC(C)c1cc(O)c(O)c(/C=C/C(=N)NCCN2CCN(CCNc3ccc4...
3,CC(=O)C1=C(C)c2cc(O)cc(/C=C/c3cc(C)c(O)c(O)c3O...
4,CCN(CC)CCN(CCO)CCNC(=O)Oc1ccc2c(c1)Nc1cc(O)ccc...
...,...
8717,CC(=O)Nc1cc(Nc2ncc(Br)c(Nc3ccccc3Br)n2)ccc1NCC...
8718,Cc1ccc(Nc2nc(Nc3cc(Cl)c(O)c(F)c3NCCN)ncc2I)c(N...
8719,OCCCCNCC(O)CNC(O)c1ccccc1Nc1ncc(Br)c(Nc2cc(F)c...
8720,O=C(CNCCCNCC(O)CO)Nc1ccc(Nc2ncc(Br)c(Nc3cc(F)c...


In [349]:
# Проверяем каждый Smiles по 6 крититериям и записываем в массив points количество успехов

fpgen = AllChem.GetMorganGenerator(radius=2)

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
catalog = FilterCatalog(params)

model = xgb.XGBRegressor()
model.load_model("xgb_model.json")
points = []
for i in range(len(df)):
    p = 0
    df['SMILES'][i]
    #1) QED критерий
    smile = df['SMILES'][i]
    mol = Chem.MolFromSmiles(smile)
    score = QED.qed(mol)
    if score >= 0.5: p += 1
    #2) SA score (синтезируемость)
    if sascorer.calculateScore(mol) > 2 and sascorer.calculateScore(mol) < 6: p += 1
    #3) Toxicophore
    matches = catalog.GetMatches(mol)
    if len(matches) == 0: p += 1
    # 4) pValue
    fingerprint = list(fpgen.GetFingerprint(mol))
    if model.predict([fingerprint]) > -3: p += 1
    # 5) Lipinski 5
    mw = Descriptors.MolWt(mol)               # молекулярная масса
    logp = Descriptors.MolLogP(mol)           # логP (жирорастворимость)
    h_donors = NumHDonors(mol)                 # число доноров H-связей
    h_acceptors = NumHAcceptors(mol)          # число акцепторов H-связей
    n_vio = 0
    if mw > 500: n_vio += 1
    if logp > 5: n_vio += 1
    if h_donors > 5: n_vio += 1
    if h_acceptors > 10: n_vio += 1
    if n_vio <= 1: p += 1 
    #6) BBB
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)
    logbb = 0.152 * logp - 0.0148 * tpsa + 0.139
    if (mw < 500 and logp > -1 and logp < 5 and hbd <= 3 and tpsa < 90) or (-1 < logbb < 0.5): p += 1
    
    points.append(p)

In [350]:
import numpy as np
points = np.array(points)
np.unique(points)

array([1, 2, 3, 4])

In [351]:
# Смотрим статистику по Smiles
scc = [0, 0, 0, 0, 0, 0, 0]
for i in range(len(points)):
    scc[points[i]] += 1
print(scc)

[0, 59, 35, 5, 1, 0, 0]


In [352]:
# Создаем новую таблицу, куда складываем 'хорошие' молекулы (удовлетворяют >= 5 критериям)
data = {
    'SMILES': [0],
    'QED': [0],
    'SA': [0],
    'Toxicophore': [0],
    'pValue': [0],
    'Lipinski violated': [0],
    'BBB': [0],
    'Num_successes': [0]
}

df_good = pd.DataFrame(data)
df_good

,SMILES,QED,SA,Toxicophore,pValue,Lipinski violated,BBB,Num_successes
0,0,0,0,0,0,0,0,0


In [353]:
df_good

,SMILES,QED,SA,Toxicophore,pValue,Lipinski violated,BBB,Num_successes
0,0,0,0,0,0,0,0,0


smile = df['SMILES'][44711]
mol = Chem.MolFromSmiles(smile)
df_good.at[0, 'SMILES'] = smile
df_good.at[0, 'QED'] = score
df_good.at[0, 'SA'] = sascorer.calculateScore(mol)
df_good.at[0, 'Toxicophore'] = 0
df_good.at[0, 'pValue'] = model.predict([fingerprint])+6
df_good.at[0, 'Lipinski violated'] =  n_vio 
df_good.at[0, 'BBB'] = '+'
df_good.at[0, 'Num_successes'] = 6

In [354]:
# Процесс добавления 'хороших' молекул

i = 0
k = 1
while i < len(points):
    if points[i] >= 5 : 
        print(i)
        smile = df['SMILES'][i]
        print(smile)
        mol = Chem.MolFromSmiles(smile)
        df_good.at[k, 'SMILES'] = smile
        #1) QED критерий
        score = QED.qed(mol)
        df_good.at[k, 'QED'] = score
        #2) SA score (синтезируемость)
        df_good.at[k, 'SA'] = sascorer.calculateScore(mol)
        #3) Toxicophore
        matches = catalog.GetMatches(mol)
        df_good.at[k, 'Toxicophore'] = len(matches)
        # 4) pValue
        fingerprint = list(fpgen.GetFingerprint(mol))
        df_good.at[k, 'pValue'] = model.predict([fingerprint])+6
        # 5) Lipinski 5
        mw = Descriptors.MolWt(mol)               # молекулярная масса
        logp = Descriptors.MolLogP(mol)           # логP (жирорастворимость)
        h_donors = NumHDonors(mol)                 # число доноров H-связей
        h_acceptors = NumHAcceptors(mol)          # число акцепторов H-связей
        n_vio = 0
        if mw > 500: n_vio += 1
        if logp > 5: n_vio += 1
        if h_donors > 5: n_vio += 1
        if h_acceptors > 10: n_vio += 1
        df_good.at[k, 'Lipinski violated'] =  n_vio 
        #6) BBB
        mw = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        hbd = Descriptors.NumHDonors(mol)
        hba = Descriptors.NumHAcceptors(mol)
        tpsa = Descriptors.TPSA(mol)
        logbb = 0.152 * logp - 0.0148 * tpsa + 0.139
        if (mw < 500 and logp > -1 and logp < 5 and hbd <= 3 and tpsa < 90) or (-1 < logbb < 0.5): df_good.at[k, 'BBB'] = '+'
        else: df_good.at[k, 'BBB'] = '-'
        df_good.at[k, 'Num_successes'] = 5
        k += 1
    i += 1


In [355]:
df_good

,SMILES,QED,SA,Toxicophore,pValue,Lipinski violated,BBB,Num_successes
0,0,0,0,0,0,0,0,0


In [356]:
# Если молекула токсичная, то удаляем её, даже если остальные 5 параметров успешные
for i in range(1, len(df_good)):
    if df_good['Toxicophore'][i] != 0: df_good = df_good.drop(i)

In [357]:
df_good

,SMILES,QED,SA,Toxicophore,pValue,Lipinski violated,BBB,Num_successes
0,0,0,0,0,0,0,0,0


In [358]:
df_good = df_good.drop(0)

In [359]:
df_good

,SMILES,QED,SA,Toxicophore,pValue,Lipinski violated,BBB,Num_successes


In [344]:
df_good.to_csv("good_mol_final.csv", index=False)